# London at Night
### Follow the Instructions here https://developers.google.com/earth-engine/guides/image_relational#colab-python_1 

In [1]:
import ee
import folium
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=p1OgBLAuCdq3EQb1r7doliO1nVG9uNiCbBCVvR15SZ8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhRi0MGk2SpNf6f6Oj0dlDs9X0mwvopufzJhiajunnx3iR0suHTjw8

Successfully saved authorization token.


In [18]:
# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [19]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# Define a map centered on SANTIAGO DE CHILE.
map_bare = folium.Map(location=[-33.43948207844584, -70.67172307031677], zoom_start=12) #-33.43948207844584, -70.67172307031677

# Add the masked image layer to the map and display it.
map_bare.add_ee_layer(bare.selfMask(), None, 'bare')
display(map_bare)

In [30]:
# Load a cloudy Sentinel-2 image.
image = ee.Image(
    'COPERNICUS/S2_SR/20210114T185729_20210114T185730_T10SEG')

# Load another image to replace the cloudy pixels.
replacement = ee.Image(
    'COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')

# Set cloudy pixels (greater than 5% probability) to the other image.
replaced = image.where(image.select('MSK_CLDPRB').gt(5), replacement)

# Define a map centered on Sgto of Chile
map_replaced = folium.Map(location=[-33.43948207844584, -70.67172307031677], zoom_start=10)

# Display the images on a map.
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}
map_replaced.add_ee_layer(image, vis_params, 'original image')
map_replaced.add_ee_layer(replaced, vis_params, 'clouds replaced')
display(map_replaced.add_child(folium.LayerControl()))

In [27]:
# Load a 2012 nightlights image.
nl_2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl_2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Define a map centered on Chile
map_zones = folium.Map(location=[-33.43948207844584, -70.67172307031677], zoom_start=9)

# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
map_zones.add_ee_layer(
    zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones')
display(map_zones)

In [28]:
# Create zones using an expression, display.
zones_exp = nl_2012.expression("(b('stable_lights') > 62) ? 3 "
                               ": (b('stable_lights') > 55) ? 2 "
                               ": (b('stable_lights') > 30) ? 1 "
                               ": 0")

# Define a map centered on Chile.
map_zones_exp = folium.Map(location=[-33.43948207844584, -70.67172307031677], zoom_start=5)

# Add the image layer to the map and display it.
map_zones_exp.add_ee_layer(
    zones_exp, {'min': 0, 'max': 3, 'palette': palette}, 'zones exp')
display(map_zones_exp)